In [1]:
import os
import sys
sys.path.append("..")

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision

from data import loader_train, loader_val, loader_test
from train import train_model
from models import get_model
from utils import show_img
from trainers import Regression

In [ ]:
# Iteración de prueba para ver imágenes y sus clasificaciones
train_loader = loader_train



#train a model
model = get_model("mobilenet_v2")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
trainer = Regression(model, device)

model.eval()
for inputs, labels in train_loader:
    inputs = inputs.to(device)
    labels = labels.to(device).double()

    outputs = model(inputs)
    print(outputs[0])
    print(labels[0].double().numpy)

    break


In [ ]:
#train a model
model = get_model("mobilenet_v2")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
trainer = Regression(model, device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_model(model, loader_train, loader_val, trainer, optimizer, device, num_epochs=10)

c:\Users\34658\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\34658\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/9
----------


Training Epoch [1/10]:  61%|████████████████████████▉                | 440/724 [06:11<04:14,  1.12it/s, curr_train_acc=0.3142, curr_train_loss=0.8992]